In [3]:
!pip install pennprov

     |████████████████████████████████| 101 kB 1.2 MB/s ta 0:00:011


In [13]:
from pennprov.connection.mprov_connection import MProvConnection
from pennprov.metadata.stream_metadata import BasicSchema, BasicTuple
from datetime import datetime, timezone

def area_circle(input):
    return {'key': input['key'],
        'value': input['value'] * input['value'] * 3.1415}

conn = MProvConnection('YOUR_USERNAME', 'YOUR_PASSWORD', 'host.docker.internal:8088')

# Create a simple relation or stream, with a binary schema
data_schema = BasicSchema('SampleStream', {'key': 'int', 'value':'int'})


In [17]:
# Create a sample tuple
for i in range(0,10):
    tuple = BasicTuple(data_schema, {'key': i, 'value': 456 + i})
    tuple_index = i

    # Store the initial data, get the ID (token) of its node in the graph
    input_token = conn.store_stream_tuple('SampleStream', tuple_index, tuple)

    # Compute an operation over the tuple, convert it to a tuple
    ts = datetime.now(timezone.utc)
    result = area_circle(tuple)
    out_tuple = BasicTuple(data_schema, result)

    # Store the derived tuple and the derivation name / time
    conn.store_derived_result('OutStream', tuple_index, out_tuple, input_token, 'area_circle', ts, ts)
